In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import itertools
import os
import shutil
import random
import timeit
import glob
import matplotlib.pyplot as plt 
import warnings
from recurrence_plot import RecurrencePlot as rp
#warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline

### Check for compatable GPU (optional)

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


### Declare global variables

In [4]:
chaos_labels = np.array([1,0,0])
period_labels = np.array([0,1,0])
trend_labels = np.array([0,0,1])

chaos_path = 'rp-data/chaos'
period_path = 'rp-data/period'
trend_path = 'rp-data/trend'

train_data_path = 'rp-data/train_data.npz'

### Transform images into pixel arrays and save them as a .npz file

In [5]:
def binarize_image_array(pixels_array):
    graph_list = []
    triangle_width = len(pixels_array) - 1

    for i in range(triangle_width):
        for j in range(triangle_width - i):
            pixel = 1
            if (pixels_array[i][j][0] == np.float32(255)):
                pixel = 0
             
            graph_list.append(pixel)
            
    return graph_list
    

def populate_train_data(image_dir, train_label):
    counter = 0
    t_start = timeit.default_timer()
    t_samples, t_labels = [], []
    
    for file in os.listdir(image_dir):
        filename = os.fsdecode(file)
        
        if filename.endswith(".png"):
            counter += 1
            
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path)
            
            pixels_array = img_to_array(img)
            pixels_array_binarized = binarize_image_array(pixels_array)
            
            t_samples.append(pixels_array_binarized)
            t_labels.append(train_label)
            
            if (counter % 50 == 0):
                print("Time taken: ", timeit.default_timer()-t_start\
                     ,'\tImages processed: ', counter)

    t_end = timeit.default_timer()
    print("Total time taken: ", t_end - t_start)
    
    t_samples = np.array(t_samples, dtype=np.uint8)
    t_labels = np.array(t_labels, dtype=np.uint8)
    
    return [t_samples, t_labels]

chaos_data = populate_train_data(chaos_path, chaos_labels)
period_data = populate_train_data(period_path, period_labels)
trend_data = populate_train_data(trend_path, trend_labels)

train_samples = np.concatenate((chaos_data[0], period_data[0], trend_data[0]), axis=0)
train_labels = np.concatenate((chaos_data[1], period_data[1], trend_data[1]))

np.savez(train_data_path, samples=train_samples, labels=train_labels)

Time taken:  9.395362567999996 	Images processed:  50
Time taken:  18.778466397999978 	Images processed:  100
Time taken:  28.339144472999976 	Images processed:  150
Time taken:  38.23711120399997 	Images processed:  200
Time taken:  47.62365780099998 	Images processed:  250
Time taken:  56.951241323999966 	Images processed:  300
Time taken:  66.25916217700001 	Images processed:  350
Time taken:  75.91146799999996 	Images processed:  400
Time taken:  85.45756617 	Images processed:  450
Time taken:  95.09342536399998 	Images processed:  500
Time taken:  104.61276306799999 	Images processed:  550
Time taken:  114.139485907 	Images processed:  600
Time taken:  123.42727572399997 	Images processed:  650
Time taken:  132.91935320699997 	Images processed:  700
Time taken:  142.330634423 	Images processed:  750
Time taken:  151.61234488000002 	Images processed:  800
Time taken:  160.893504162 	Images processed:  850
Time taken:  170.76986989499994 	Images processed:  900
Time taken:  180.2662

### Load train data into numpy arrays

In [5]:
train_data = np.load(train_data_path)
train_samples = train_data['samples']
train_labels = train_data['labels']

### Assign data into "train", "valid" and "test" groups

In [50]:
print(len(train_samples[0]))

for i in train_samples:
    if (len(i) != 258840):
        print('Bad len')

258840


### Shuffle the data

In [7]:
train_labels, train_samples = shuffle(train_labels, train_samples)

### Define data model

In [40]:
model = Sequential([
    Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(258840,1)),
    Conv1D(filters=64, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=1),
    Dropout(0.2),
    Conv1D(filters=32, kernel_size=1, activation='relu'),
    Conv1D(filters=32, kernel_size=1, activation='relu'),
    MaxPooling1D(pool_size=1),
    Dropout(0.5),
    Flatten(),
#     Dense(8, activation='relu'),
    Dense(units=3, activation='softmax')
])

In [41]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_65 (Conv1D)           (None, 258840, 64)        128       
_________________________________________________________________
conv1d_66 (Conv1D)           (None, 258840, 64)        4160      
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 258840, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 258840, 64)        0         
_________________________________________________________________
conv1d_67 (Conv1D)           (None, 258840, 32)        2080      
_________________________________________________________________
conv1d_68 (Conv1D)           (None, 258840, 32)        1056      
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 258840, 32)      

### Compile the model

In [44]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
for layer in model.layers:
    print(layer.name)

conv1d_65
conv1d_66
max_pooling1d_23
dropout_5
conv1d_67
conv1d_68
max_pooling1d_24
dropout_6
flatten_20
dense_41


### Train the model

In [43]:
model.fit(x=train_samples, y=train_labels, epochs=50, verbose=2)

Epoch 1/50


ValueError: in user code:

    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/coderius/.local/lib/python3.8/site-packages/tensorflow/python/k

In [29]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adam(learning_rate=0.0001),
#              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

# Predict

In [16]:
predictions = model.predict(x=test_batches, verbose=0)

In [17]:
np.round(predictions)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.